In [1]:
import pandas as pd

In [39]:
class designFeature():
    def changePosition(self, data, key1, key2):
        storage = pd.DataFrame(data[key1])
        data.loc[:, key1] = data[key2].values
        data.loc[:, key1] = storage
        return data
    
    def changeEntrys(self, data):
        return [data.squeeze()[::-1]]
    
    def swapInfo(self, data):
        data.FinalResult = self.changeEntrys(data.FinalResult)
        data.HalftimeResult = self.changeEntrys(data.HalftimeResult)
        data = self.changePosition(data, 'GoalTimeTeam1', 'GoalTimeTeam2')
        data = self.changePosition(data, 'OvertimeGoalTeam1', 'OvertimeGoalTeam2')
        data = self.changePosition(data, 'OwnGoalTeam1', 'OwnGoalTeam2')
        data = self.changePosition(data, 'PeanltyTeam1', 'PeanltyTeam1')
        data = self.changePosition(data, 'ScorerTeam1', 'ScorerTeam2')
        return data
    
    def featureMatchday(self, data):
        matchdayData = pd.DataFrame()
        for string, Data in zip(['final', 'halftime'], [data.FinalResult, data.HalftimeResult]):
            matchdayData[string + 'GoalDiff'] = np.diff(Data.squeeze())
            matchdayData[string + 'scoredGoal'] = Data.squeeze()[0]
            matchdayData[string + 'GetGoal'] = Data.squeeze()[1]
        return matchdayData

In [58]:
class genFeature(designFeature):
    def __init__(self, year, day):
        self.genMatchday = [year, day]
        self.dataActuellMatchday = self.loadMatchday(*self.genMatchday)
        
    def loadMatchday(Matchday, year, day):
        '''Load Matchday with all paarings'''
        return pd.read_pickle('./data/'+str(year)).query('Matchday=='+str(day))
    
    def dataOfATeam(self, data, team):
        ''' Search Match of special team and return data and look if home team 
        Parameters:
            data: dict
                Matchday data with converted data
            team: string 
                Teamname of the Team
        Returns:
            list: [isHomeTeam, dataTeam]
                isHomeTeam: boolean if the searched team plays home
                dataTeam: reduced data to only the match the team played
        '''
        mask = np.array([])
        for x in ['Team1','Team2']:
            inData = [team] == data[x].values
            mask = np.append(mask, inData) # boolean of wher is searched team
        mask = mask.reshape(2, int(len(mask)/2)) # reduce into two dim. [home, away]
        boolHome = np.sum(mask, axis=1)[0].astype(bool) # return true if searched team is home Team
        return [boolHome, data[np.sum(mask, axis=0).astype(bool)]]
    
    def unifyData(self, home, data):
        ''' unify home/away data, fill empty arrays with Nones
        and add boolean Feature 'home'''
        for x in data.keys():
            if(data[x].squeeze().size==0):
                data[x]= np.array([None])
        if(home!=True):
            data = designFeature().swapInfo(data)
            data['home'] = False
        else:
            data['home']=True
        return data
    
    def featureMatchday(self, data):
        return designFeature().featureMatchday(data)
        
    def appendInfo(self, team, info, iteration):
        return self.dataActuellMatchday#['Team1' == team]

In [59]:
testMatchday = genFeature(2017,2).appendInfo('Hertha BSC',1,1)

In [60]:
testTeam = genFeature(2017,2).dataOfATeam(testMatchday, 'Hertha BSC')

In [61]:
testTeam[1].keys()


Index(['DateTime', 'FinalResult', 'GoalTimeTeam1', 'GoalTimeTeam2',
       'HalftimeResult', 'Hour', 'Matchday', 'OvertimeGoalTeam1',
       'OvertimeGoalTeam2', 'OwnGoalTeam1', 'OwnGoalTeam2', 'PeanltyTeam1',
       'PenaltyTeam2', 'ScorerTeam1', 'ScorerTeam2', 'Spectators', 'Team1',
       'Team2', 'Weekday'],
      dtype='object')

In [62]:
data = genFeature(2017,2).unifyData(*testTeam)

/home/maximilian/.local/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/maximilian/.local/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/home/maximilian/.local/anaconda3/lib/python3.5/site-packages/pandas/core/indexing.py:476: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in